Up until now, data has been downloaded and preprocessed to remove unncessary columns and features. then, a set of rules-based regex functions were created as soft labels for common restricted ingredient classes (allergens, cultural restrictions, etc). Now, Uthese functions will be used to seed-label a subset fo the cleaned training examples (~20,000), whichwill eventually be used to train an XGBoost algorithm to mroe robustly classify all classes. This will allow for all data to be flexibly labeled, creating a high qualiuty training set for LLM fine tuning. 